# **Loan Default Prediction**

This notebook demonstrates **Exploratory Data Analysis (EDA)** on a loan dataset with a **binary target** (`bad_flag`), followed by training a **PyTorch neural network** to predict whether a loan will default.

## **Notebook Outline**
1. **Imports & Setup**
2. **Data Loading and Initial Inspection**
3. **Data Cleaning & Preprocessing**
4. **Exploratory Data Analysis (EDA)**
5. **Feature Selection & Final Preparation**
6. **Train-Test Split & Imbalance Handling**
7. **PyTorch Model Building & Training**
8. **Evaluation & Threshold Tuning**

We'll see both **Markdown explanations** (like this one) and **Code cells** demonstrating each step.

---

In [1]:
# (Cell 1) 1. Imports & Setup
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (confusion_matrix, precision_score, recall_score, 
                             f1_score, roc_auc_score, classification_report)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import RandomOverSampler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

plt.style.use('seaborn')  # optional aesthetics
print("Setup complete.")

Setup complete.


<ipython-input-1-7137d4bfe437>:23: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')  # optional aesthetics


## **2. Data Loading and Initial Inspection**
We assume you have a CSV file named `training_loan_data.csv`. Adjust `header` or `skiprows` as needed.

In [3]:
# (Cell 2) 2. Data Loading and Quick Look
df = pd.read_csv('training_loan_data.csv', header=1)  # Adjust path if necessary
print("Data loaded. Shape:", df.shape)
df.head()

Data loaded. Shape: (199121, 23)


,id,member_id,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,desc,purpose,...,inq_last_6mths,mths_since_recent_inq,revol_util,total_bc_limit,mths_since_last_major_derog,tot_hi_cred_lim,tot_cur_bal,application_approved_flag,internal_score,bad_flag
0,10000001,11983056.0,7550,36 months,16.24%,3 years,RENT,28000.0,NaN,debt_consolidation,...,0.0,17.0,72%,4000.0,NaN,3828.953801,5759.0,1,99,0.0
1,10000002,12002921.0,27050,36 months,10.99%,10+ years,OWN,55000.0,Borrower added on 12/31/13 > Combining high ...,debt_consolidation,...,0.0,8.0,61.20%,35700.0,NaN,34359.940730,114834.0,1,353,0.0
2,10000003,11983096.0,12000,36 months,10.99%,4 years,RENT,60000.0,Borrower added on 12/31/13 > I would like to...,debt_consolidation,...,1.0,3.0,24%,18100.0,NaN,16416.617760,7137.0,1,157,0.0
3,10000004,12003142.0,28000,36 months,7.62%,5 years,MORTGAGE,325000.0,NaN,debt_consolidation,...,1.0,3.0,54.60%,42200.0,NaN,38014.149760,799592.0,1,365,0.0
4,10000005,11993233.0,12000,36 months,13.53%,10+ years,RENT,40000.0,NaN,debt_consolidation,...,0.0,17.0,68.80%,7000.0,53.0,6471.462236,13605.0,1,157,0.0


## **3. Data Cleaning & Preprocessing**
Here, we:
1. Convert textual columns (`term`, `int_rate`, `revol_util`, `emp_length`) to numeric.
2. Drop rows missing `bad_flag` (the target), since they can't be used for supervised learning.
3. Drop or impute columns with missing data.
4. One-hot encode categorical columns (`home_ownership`, `purpose`).
5. Remove ID-like or leak-prone columns (`id`, `member_id`, `application_approved_flag`).

In [4]:
# (Cell 3) Data Cleaning Functions

def convert_emp_length(x):
    """Converts strings like '10+ years', '3 years', '< 1 year' to integers."""
    if pd.isnull(x):
        return np.nan
    x = str(x).lower()
    if '10+' in x:
        return 10
    elif '< 1' in x:
        return 0
    match = re.search(r'(\d+)', x)
    if match:
        return int(match.group(1))
    return np.nan

# Convert 'term' from '36 months' -> 36
if 'term' in df.columns:
    df['term'] = df['term'].str.replace(' months', '', regex=False)
    df['term'] = pd.to_numeric(df['term'], errors='coerce')

# Convert percentage columns
for col in ['int_rate', 'revol_util']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace('%', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert emp_length
if 'emp_length' in df.columns:
    df['emp_length'] = df['emp_length'].apply(convert_emp_length)

print("After basic conversions:")
print(df.info())

# Drop rows without bad_flag
df = df.dropna(subset=['bad_flag'])

# Drop textual 'desc' if it exists
if 'desc' in df.columns:
    df.drop(columns=['desc'], inplace=True)

# Impute special columns
if 'mths_since_last_major_derog' in df.columns:
    df['mths_since_last_major_derog'].fillna(999, inplace=True)
if 'mths_since_recent_inq' in df.columns:
    df['mths_since_recent_inq'].fillna(36, inplace=True)

# Impute numeric columns with median
num_impute_cols = [
    'tot_cur_bal', 'bc_util', 'percent_bc_gt_75', 
    'emp_length', 'tot_hi_cred_lim', 'total_bc_limit', 
    'revol_util', 'dti', 'inq_last_6mths', 
    'annual_inc', 'int_rate', 'term'
]
for col in num_impute_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

# One-hot encode selected categorical columns
categorical_cols = ['home_ownership', 'purpose']
for c in categorical_cols:
    if c in df.columns:
        # drop_first=True to avoid dummy trap
        df = pd.get_dummies(df, columns=[c], drop_first=True)

# Drop ID-like or leak columns
drop_cols = ['id', 'member_id', 'application_approved_flag']
df.drop(columns=drop_cols, errors='ignore', inplace=True)

print("Final shape after cleaning:", df.shape)
df.head()

After basic conversions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199121 entries, 0 to 199120
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           199121 non-null  int64  
 1   member_id                    189457 non-null  float64
 2   loan_amnt                    199121 non-null  int64  
 3   term                         189457 non-null  float64
 4   int_rate                     189457 non-null  float64
 5   emp_length                   181531 non-null  float64
 6   home_ownership               189457 non-null  object 
 7   annual_inc                   189457 non-null  float64
 8   desc                         82004 non-null   object 
 9   purpose                      189457 non-null  object 
 10  percent_bc_gt_75             180419 non-null  float64
 11  bc_util                      180333 non-null  float64
 12  dti                          1894

,loan_amnt,term,int_rate,emp_length,annual_inc,percent_bc_gt_75,bc_util,dti,inq_last_6mths,mths_since_recent_inq,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,7550,36.0,16.24,3.0,28000.0,100.0,96.0,8.40,0.0,17.0,...,False,False,False,False,False,False,False,False,False,False
1,27050,36.0,10.99,10.0,55000.0,25.0,53.9,22.87,0.0,8.0,...,False,False,False,False,False,False,False,False,False,False
2,12000,36.0,10.99,4.0,60000.0,0.0,15.9,4.62,1.0,3.0,...,False,False,False,False,False,False,False,False,False,False
3,28000,36.0,7.62,5.0,325000.0,16.7,67.1,18.55,1.0,3.0,...,False,False,False,False,False,False,False,False,False,False
4,12000,36.0,13.53,10.0,40000.0,33.3,79.6,16.94,0.0,17.0,...,False,False,False,False,False,False,False,False,False,False
